In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/03/25 19:29:25 WARN Utils: Your hostname, MacBook-Air-4.local resolves to a loopback address: 127.0.0.1; using 10.0.0.136 instead (on interface en0)
24/03/25 19:29:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 19:29:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [4]:
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76')

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
start_time = time.time()

print((time.time()-start_time))



2.8133392333984375e-05


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = spark.sql("""SELECT
                        YEAR(date) AS year,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     Where
                         bedrooms = 4
                     GROUP BY
                         year
                     ORDER BY
                         year""")

query.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query2 = spark.sql("""SELECT
                        YEAR(date_built) AS year,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     Where
                         bedrooms = 3
                         AND bathrooms = 3
                     GROUP BY
                         year
                     ORDER BY
                         year""")

query2.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query3 = spark.sql("""SELECT
                        YEAR(date_built) AS year,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     Where
                         bedrooms = 3
                         AND bathrooms = 3
                         AND floors = 2
                         AND sqft_living >= 2000
                     GROUP BY
                         year
                     ORDER BY
                         year""")

query3.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [19]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = spark.sql("""SELECT
                        view,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     GROUP BY
                         view
                     HAVING AVG(price) >= 35000
                     ORDER BY
                         view desc""")
query4.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 282001.65|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6956908702850342 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query4 = spark.sql("""SELECT
                        view,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     GROUP BY
                         view
                     HAVING AVG(price) >= 35000
                     ORDER BY
                         view desc""")
query4.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 282001.65|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.2257699966430664 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('homes_parquet', mode = 'overwrite')

In [24]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('homes_parquet')

In [25]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_homes')

In [26]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query4 = spark.sql("""SELECT
                        view,
                        ROUND(AVG(price),2) AS avg_price
                     From
                         home_sales
                     GROUP BY
                         view
                     HAVING AVG(price) >= 35000
                     ORDER BY
                         view desc""")
query4.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 282001.65|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.20487713813781738 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [28]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sales"):
  print("table is till cached")
else:
  print("all clear")

all clear
